<b>don't forget! 👇</b>

In [4]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


### Загрузка необходимых библиотек

In [2]:
#must have
import pandas as pd
import numpy as np

# stats and tests
from scipy import stats as st

# visualisation
from matplotlib import pyplot as plt
import seaborn as sns

# hash
import hashlib
# it's booster for apply
import swifter

# additional
import random
import math
import datetime
import ipywidgets

# working with json format
import json

# map, choropleth and marker
from folium import Map, Choropleth, Marker
# clusters
from folium.plugins import MarkerCluster

#### Настройка оформления вывода (out)

In [ ]:
# вывод всех столбцов
pd.set_option('display.max_columns', None)
# отражение текста в ячейке полностью вне зависимости от длины
pd.set_option('display.max_colwidth', None)
# волшебная команда для подробного отображения ошибок
%xmode Verbose
# отображение графиков в ячейках блокнота
%matplotlib inline

### Создание необходимых функций для работы

##### describe_data

In [ ]:
def describe_data(data):
    """
     Выводит основные характеристики DataFrame, включая:
    - 10 случайных строк из таблицы
    - размерность DataFrame
    - информацию по числовым столбцам
    - информацию по строковым столбцам
    - количество пропусков в таблице
    - количество уникальных значений в каждом столбце
    - количество дублирующихся строк
    - количество уникальных значений в каждом столбце
    """
    # вывод 10 случайных строк
    print('\033[92mвывод случайных строк из таблицы:\033[90m\n')
    # если строк в таблице меньше 10, то выскочит ошибка
    # отлавливаем ее и выводим все имеющиеся строки в таблице
    try:
        display(data.sample(10))
    except:
        display(data.sample(data.shape[0]))

    # вывод типов данных в столбцах
    print('Типы данных:')
    print(data.dtypes)
    ('-----------------------------------------------------------------------')

    # вывод количества строк
    print('\033[92mКоличество строк:\033[90m',
          f'\033[91m{data.shape[0]}\033[90m')
    print('-----------------------------------------------------------------------')

    # вывод количества столбцов
    print('\033[92mКоличество столбцов:\033[90m',
          f'\033[91m{data.shape[1]}\033[90m')
    print('-----------------------------------------------------------------------')

    # вывод информации о числовых столбцах
    print('\033[92mИнформация по числовым столбцам:\033[90m\n')
    for column in data.select_dtypes(include='number'):
        print(data[column].describe(), '\n')
        print('-----------------------------------------------------------------------')

    # вывод информации об оъектных данных
    print('\033[92mИнформация по строковым столбцам:\033[90m\n')
    for column in data.select_dtypes(include='object'):
        print(data[column].describe(), '\n')
        print('-----------------------------------------------------------------------')

    # вывод столбцов, содержащих пропуски, и их количество
    print('\033[92mКоличество пропусков в таблице:\033[90m\n')
    # эта переменная нужно для подсчета столбиков с пропусками
    count_na = 0
    for column in data.columns:
        null_in_column = data[column].isna().sum()

        if null_in_column:
            count_na += 1
            print(f'\033[91m{column}: {null_in_column}\033[90m')
            print(
                '-----------------------------------------------------------------------')
    if count_na == 0:
        print('В таблице нет пропусков 🥂')

    # вывод информации об уникальных значениях каждого столбца
    print('\033[92mКоличество уникальных значений в каждом столбце:\033[90m\n')
    for column in data.columns:
        unique_values = data[column].nunique()
        print(f'{column} : {unique_values}')
        print('-----------------------------------------------------------------------')

    # вывод информации о дублирующих строках
    print('\n')
    duplicated_value = data.duplicated().sum()
    if duplicated_value:
        print(
            f'Количество дублирующихся строк: \033[91m{duplicated_value}\033[90m')
    else:
        print('\033[94mДублирующихся строк не обнаружено\033[90m')
    print('\n')

##### create_missing_df

In [7]:
def create_missing_df(data):
    """
    функция, которая сформирует нам таблицу с количеством пропусков и вернет ее
    на вход принимает только один параметр - DataFrame
    """
    # формируем дф с количеством пропусков в столбцах
    # считаем количество пропусков и сбрасываем индексы
    missing_df = data.isna().sum(axis=0).reset_index()

    # переименуем колонки
    missing_df.columns = ['column_name', 'missing_count']

    # оставляем только те строки, в столбцах которых пристуствуют пропуски + сортировка
    missing_df = missing_df[missing_df['missing_count'] > 0]
    missing_df = missing_df.sort_values('missing_count', ascending=True)

    # считаем % пропусков от общего
    missing_df['%'] = np.round((missing_df['missing_count'] / data.shape[0])*100, 2)

    # возвращаем дф с количеством пропусков в каждом столбце
    return missing_df

##### remove_outliners_IQR

In [ ]:
def remove_outliners_IQR(data, column_name):
    """
    Функция использует межквартильный диапазон для удаления выбросов из DataFrame,
    создает графики до и после удаления выбросов,
    возвращает обновленный DataFrame и DataFrame с аномалиями.
    Параметры:
    - data: исходные данные.
    - column_name: наименование столбца данных.
    """
    # создаем копию таблицы
    data_after_remove = data.copy()
    # вычисляем статистические характеристики столбца (квартили)
    Q1 = data_after_remove[column_name].quantile(0.25)
    Q3 = data_after_remove[column_name].quantile(0.75)
    IQR = Q3 - Q1
    # вычисляем границы межквартильного диапазона
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # удаляем выбросы из столбца
    # запишем удаленные строчки в новую таблицу
    removed_df = data_after_remove[(data_after_remove[column_name] < lower_bound) |
                                   (data_after_remove[column_name] > upper_bound)].copy()

    # перезапишем основную таблицу
    data_after_remove = data_after_remove[(data_after_remove[column_name] >= lower_bound) &
                                          (data_after_remove[column_name] <= upper_bound)]

    # визуализируем графики до и после
    plt.figure(figsize=(12, 6))

    # первый график
    plt.subplot(1, 2, 1)
    plt.boxplot(data[column_name])
    plt.title('До удаления выбросов')
    plt.ticklabel_format(style='plain', axis='y')

    # второй график
    plt.subplot(1, 2, 2)
    plt.boxplot(data_after_remove[column_name].dropna())
    plt.title('После удаления выбросов')
    plt.ticklabel_format(style='plain', axis='y')

    # возвращаем обновленный дф и дф с аномалиями
    return data_after_remove, removed_df

##### create_random_color

In [ ]:
def create_random_color():
    """
    Функция генерирует случайный цвет 
    """
    r = random.random()
    g = random.random()
    b = random.random()
    return (r, g, b)

##### convert_to_lowercase

In [1]:
def convert_to_lowercase(data):
    
    """
    Функция приводит к нижнему регистру и к змеиной нотации значения столбцов и строк из DataFrame,
    в наименованиях столбцов заменяет пробелы на андерскор,
    возвращает обновленный DataFrame
    Параметры:
    - data: исходные данные.
    """
    def camel_to_snake(word):
        """
        функция для конвертирования нотации camelCase к snake_case
        """
        
        snake_case_string = ""
        for i, c in enumerate(word):
            if i == 0:
                snake_case_string += c.lower()
            elif c.isupper():
                snake_case_string += "_" + c.lower()
            else:
                snake_case_string += c
        return snake_case_string
    
    # приведение значения столбцов к нижнему регистру и змеиной нотации
    data.rename(columns=camel_to_snake, inplace=True)
    
    # работа со значениями в строчках только с типом данных object
    for column in data.select_dtypes(include='object'):
        data[column] = data[column].apply(camel_to_snake)
    
    # возвращаем обновленную таблицу
    return data

##### get_color

In [3]:
def get_color(name, number):
    """
    Функция для получения различных цветов
    на вход принимает расцветку(name) и количество необходимых цветов(number)
    
    Наименования моих палеток:
    - viridis_r
    - plasma_r
    - Spectral
    - hsv
    """
    
    pal = list(sns.color_palette(palette=name, n_colors=number).as_hex())

    return pal

##### ab_split

In [2]:
def ab_split(id, salt='exp_mess_1', n_groups=5):
    """
    функция реализующая хеширование и разбиение пользователей на n групп
    """
    test_id = str(id) + '-' + str(salt)
    test_id_digest = hashlib.md5(test_id.encode('ascii')).hexdigest()
    test_id_final_int = int(test_id_digest, 16)

    return test_id_final_int % n_groups

##### prioritize_hypotheses

In [ ]:
def prioritize_hypotheses(data, method='ice'):
    """
    Функция, приоритизирующая гипотезы методом ICE или RICE,
    создает необходимые столбцы (в зависимости от выбранного метода) 
    и сортирует по убыванию DataFrame по рангу набранных баллов.
    Возвращает обновленный DataFrame.
    Параметры:
    - data: исходные данные;
    - method: метод использующийся для приоретизации гипотез (по-умолчанию ICE).
    
    -------
    ВАЖНО!!! DataFrame должен содержать необходимые столбцы, а именно:
    - impact (влияние);
    - confidence (уверенность);
    - efforts (усилия);
    - reach (охват).
    """
    
    if method == 'ice':
        # считаем и округляем до 3 цифр после запятой
        data['ice_score'] = ((data.impact * data.confidence) / data.efforts).round(3)
        
        # ранжируем для наглядности + преобразуем ранк в целое число
        data['ice_rank'] = data['ice_score'].rank(ascending=False).astype(int)
        
        # возвращаем обновленный DataFrame с сортировкой по рангу
        return data.sort_values(by='ice_rank')
    
    elif method == 'rice':
        # считаем и округляем до 3 цифр после запятой
        data['rice_score'] = (data.reach * data.impact * data.confidence) / data.efforts
        
        # ранжируем для наглядности + преобразуем ранк в целое число
        data['rice_rank'] = data['rice_score'].rank(ascending=False).astype(int)
        
        # возвращаем обновленный DataFrame с сортировкой по рангу
        return data.sort_values(by='rice_rank')
    else:
        print('Ошибка! нет такого метода')
        return data

##### get_profiles

In [ ]:
def get_profiles(sessions, 
                 orders,  
                 ad_costs
                ):
    """
    Функция предназначена для создания таблицы с профилем клиентов. Она принимает следующие параметры:
    orders: датафрейм с заказами.
    ad_costs: датафрейм с затратами на рекламу.
    """

    # сортируем сессии по ID пользователя и дате привлечения
    # группируем по ID и находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
         # время первого посещения назовём first_ts
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()  # возвращаем user_id из индекса
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())


    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
         # столбец с числом пользователей назовём unique_users
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()  # возвращаем dt и channel из индексов
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    # по дате и каналу привлечения
    ad_costs = ad_costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    # результаты сохраним в столбец acquisition_cost (CAC)
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # добавим стоимость привлечения в профили
    profiles = profiles.merge(
        ad_costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left'
    )

    # органические пользователи не связаны с данными о рекламе,
    # поэтому в столбце acquisition_cost у них значения NaN
    # заменим их на ноль, ведь стоимость привлечения равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)
    
    return profiles  # возвращаем профили с CAC

##### get_retention

In [ ]:
def get_retention(profiles, 
                  sessions, 
                  observation_date, 
                  horizon_days, 
                  dimensions = [], 
                  ignore_horizon = False
                 ):
    
    # решаем какие столбцы оставим для группировки удержания, столбец признака платящего пользователя остается всегда
    dimensions = ['payer'] + dimensions
    
    # оттсекаем всех пользователей, которые не могли "дожить" до нужного горизонта 
    # (присоединились позже чем observation_date - horizon)
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(days = horizon_days - 1)
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # формируем массив сырых данных
    result_raw = result_raw.merge(sessions[['user_id', 'session_start']], on = 'user_id', how = 'left')
    result_raw['lifetime'] = (result_raw['session_start'] - result_raw['first_ts']).dt.days
    
    # функция группировки по нужному набору измерений
    def group_by_dimensions(df, dims, horizon_days):     
        result = df.pivot_table(index = dims, columns = 'lifetime', values = 'user_id', aggfunc = 'nunique')     # строим "треугольную таблицу" 
        cohort_sizes = df.groupby(dims).agg({'user_id': 'nunique'}).rename(columns = {'user_id': 'cohort_size'}) # определяем размеры когорт
        result = cohort_sizes.merge(result, on = dims, how = 'left').fillna(0)                                   # присоединяем размеры когорт к треугольной таблице
        result = result.div(result['cohort_size'], axis = 0)                                                     # делим каждый из столбцов на размер когорты - расчитываем % удержания (retention rate)
        result = result[['cohort_size'] + list(range(horizon_days))]                                             # оставляем только наблюдения до нужного горизонта и столбец размеров когорт
        result['cohort_size'] = cohort_sizes                                                                     # пересприсваиваем столбец размеров когорт, ведь во время деления он превратился в 1
        return result
    
    # расчитываем удержание 
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)
    
    # рассчитываем удержание по всем параметрам + дата
    result_in_time = group_by_dimensions(result_raw, dimensions + ['dt'], horizon_days)
    
    # возвращаем и таблицы удержания и сырые данные (чтобы в случае чего можно было в них покопаться при отладке)
    return result_raw, result_grouped, result_in_time

##### get_ltv

In [ ]:
def get_ltv(profiles, 
            purchases,
            observation_date, 
            horizon_days,
            dimensions=[], 
            ignore_horizon=False,
           ):
    
    """
    Функция предназначена для расчета пожизненной ценности (LTV) клиентов. Она принимает следующие параметры:
    profiles: датафрейм с данными о профилях пользователей.
    purchases: датафрейм с данными о покупках пользователей.
    observation_date: дата, по состоянию на которую рассчитывается LTV.
    horizon_days: горизонт анализа в днях.
    dimensions: список дополнительных измерений, по которым будет группироваться LTV.
    ignore_horizon: флаг, указывающий, учитывать ли горизонт анализа при расчете LTV.
    """
    
    # исключаем пользователей, не «доживших» до горизонта анализа
    # Функция принимает на вход дату наблюдения observation_date, которая определяет конец горизонта анализа. 
    # Для каждого пользователя, который совершил первую покупку после observation_date, мы считаем, что его LTV равен нулю.
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    # Функция query() используется для фильтрации данных датафрейма profiles 
    # по условию dt <= @last_suitable_acquisition_date, 
    # где last_suitable_acquisition_date - дата, по состоянию на которую рассчитывается LTV.
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')
    
    # добавляем данные о покупках в профили
    # Для вычисления LTV нам необходимо знать, когда пользователь совершил первую покупку и сколько раз он покупал после этого. 
    # Для этого мы объединяем таблицы profiles и purchases по полю user_id
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    
    # рассчитываем лайфтайм пользователя для каждой покупки
    # Лайфтайм пользователя - это период времени с момента его первой покупки до момента совершения последней. 
    # Рассчитываем лайфтайм как разницу между датами покупки и привлечения пользователя.
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    
    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция группировки по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        """
        Функция group_by_dimensions принимает на вход таблицу (в нашем случае result_raw)
        horizon_days: горизонт анализа в днях.
        dimensions: список дополнительных измерений, по которым будет группироваться LTV 
        
        и возвращает две таблицы:

        result - таблица LTV, в которой каждая строка соответствует когорте и лайфтайму. 
        В ячейках таблицы содержится суммарная выручка от пользователей, совершивших покупку в этой когорте и за этот лайфтайм.
        
        roi - таблица ROI, в которой каждая строка соответствует когорте и лайфтайму. 
        В ячейках таблицы содержится отношение LTV к CAC.
        """
        
        # строим «треугольную» таблицу выручки
        result = df.pivot_table(index=dims, 
                                columns='lifetime', 
                                values='revenue', 
                                aggfunc='sum'
                               )
        
        # находим сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)
        
        # вычисляем размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        
        # объединяем размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        
        # считаем LTV: делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes

        # сохраняем в датафрейм данные пользователей и значения CAC, 
        # добавив параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []
    
    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    )